# La présence de bibliothèques autour des lycées augmente-t-elle la performance des lycées ?

In [ ]:
# Setup

# Si la commande n'avait pas déjà été lancée danas le terminal
# !pip install -r "requirements.txt"

import numpy as np
import pandas as pd
import geopandas as gpd
from src.download_file import download_file
from src.data_lycees import get_data_lycees
from src.data_lycees import merge_data_lycees
from src.data_biblio import get_data_biblio
from src.distances import compute_biblio_distances
from plotnine import *
import folium

pd.set_option('display.width', 120)

### Contexte

La performance des lycées est ici entendue comme le taux de réussite à l’examen du 
baccalauréat général, même si l’étude du taux de mention obtenu par les lycées est 
une piste envisagée. Celle-ci peut être expliquée par trois facteurs :

-	par le public de l’établissement (les élèves et leur origine sociale)
-	par le contexte de l’établissement (ses propriétés intrinsèques et possiblement les politiques 
    éducatives dont il fait objet)
-	par l’environnement de l’établissement (ce qui est relatif à sa situation géographique)

La question que l’on se pose se concentre sur les facteurs environnementaux de réussite scolaire, 
en proposant l’analyse de l’impact de la présence de bibliothèques autour des lycées. Les 
bibliothèques, qu’elles soient municipales, universitaires ou d’une autre nature, sont en effet 
des lieux qui peuvent favoriser la réussite des élèves par au moins deux moyens : d’abord, ce 
sont des lieux d’accès à la culture et à la connaissance ; d’autre part, ce sont aussi des 
lieux de révision privilégiés, notamment pour la préparation du baccalauréat .

La démarche adoptée consiste donc principalement à étudier le lien entre proximité d’une (ou de) 
bibliothèque(s) et performance des lycées. Cette proximité sera évaluée par la distance 
géographique, ou aussi par le nombre d’équipements disponibles dans un rayon donné. 


### Méthodes

@bourdieuHeritiersEtudiantsCulture1964

Après quelques statistiques descriptives et cartes pour illustrer le problème, des régressions 
linéaires seront mobilisées pour tester notre hypothèse, en contrôlant par d’autres variables. En 
partant des facteurs mis en évidence précédemment, la composition sociale du lycée sera appréciée 
à partir de l’indice de position sociale proposée par la Depp (cf. rubrique « Données »). Le 
contexte de l’établissement sera surtout pris en compte par le secteur de l’établissement 
(privé / public) et les filières et formations disponibles en son sein (filières technologiques 
et professionnelles, formations post-baccalauréat comme les BTS ou CPGE).

### Données

Pour ce qui est des données mobilisées, l’échelle d’observation sera le lycée. Le champ d’étude 
est limité aux lycées avec au moins 50 candidats (seuil à préciser) présents aux épreuves du 
baccalauréat général en 2024. Il s’agira alors de construire nos observations à partir de 
trois bases de données ouvertes produites par la Depp (service statistique ministériel de 
l’Education nationale) :
-	taux de réussite et de mention au bac (https://data.education.gouv.fr/explore/dataset/fr-en-indicateurs-de-resultat-des-lycees-gt_v2/information/)
-	indice de position sociale (https://data.education.gouv.fr/explore/dataset/fr-en-ips-lycees-ap2023/information/)
-	géolocalisation des lycées (https://data.education.gouv.fr/explore/dataset/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre/information/)

Enfin, le choix de se limiter à 2024 est lié à la disponibilité des données concernant les 
bibliothèques. En effet, nous utilisons la base permanente des équipements, qui n’est disponible 
en open data que pour 2024 (https://www.data.gouv.fr/datasets/base-permanente-des-equipements-1/).


## Analyse descriptive des données

### Les données sur les lycées

In [ ]:
# Importation des données sur les lycées

lycees_resultats, lycees_ips, annuaire_education = get_data_lycees()
lycees_data = merge_data_lycees(lycees_resultats, lycees_ips, annuaire_education)
lycees_data.info()

In [ ]:
lycees_stat_desc_num = lycees_data.melt(
    id_vars = ['uai'],
    value_vars = [
        'taux_reu_gnle', 'va_reu_gnle', 'taux_men_gnle', 'va_men_gnle',
        'ips_voie_gt', 'ecart_type_voie_gt'
    ]
)#.sort_values('uai').reset_index()

lycees_stat_desc_num['variable'] = pd.Categorical(
    lycees_stat_desc_num['variable'],
    ordered=True,
    categories=[
        'taux_reu_gnle', 'taux_men_gnle', 'ips_voie_gt', 
        'va_reu_gnle', 'va_men_gnle', 'ecart_type_voie_gt'
    ]
).rename_categories([
    'Taux de réussite', 'Taux de mention', 
    'Indice de position sociale', 'Valeur ajoutée du\ntaux de réussite', 
    'Valeur ajoutée du\ntaux de mention', 'Ecart-type de l\'IPS'
])

(
    ggplot(lycees_stat_desc_num) +
    aes(x = 'value') +
    geom_density() +
    facet_wrap('variable', scales ='free') +
    theme_bw()
)

### Les données sur les bibliothèques

In [ ]:
# Importation des données sur les bibliothèques

biblio = get_data_biblio()
biblio.info()

### Croiser les données des lycées et des bibliothèques

Après avoir considéré séparément les données des lycées et des bibliothèques, il faut maintenant les
faire se rencontrer. Une première façon de le faire est de proposer une interprétation graphique du 
problème soulevé par ce projet. Pour cela, illustrons-le à l'aide d'une carte où sont représentés à
la fois les lycées et les bibliothèques. Par simplicité, nous nous concentrerons ici sur le cas de 
la ville de Paris.

In [ ]:
lycees_75 = lycees_data[lycees_data['code_departement'] == "075"]
biblio_75 = biblio[biblio['CODEPOSTAL'].str[0:2] == "75"]

center = [biblio_75.LATITUDE.mean(), biblio_75.LONGITUDE.mean()]
m = folium.Map(
    location=center, 
    zoom_start=12
    )

tooltip = folium.GeoJsonTooltip(
    fields=['NOMETAB', 'TYPEETABABES_d', 'CONDITIONACCES_d'],
    aliases=['Nom :', 'TYPEETABABES : ', 'CONDITIONACCES_d :'],
    localize=True
)
folium.GeoJson(
    biblio_75,
    tooltip=tooltip, 
    marker=folium.Marker(icon=folium.Icon(color="green")),
).add_to(m)

tooltip = folium.GeoJsonTooltip(
    fields=['libelle_etablissement', 'presents_gnle', 'taux_reu_gnle', 'taux_men_gnle', 'ips_voie_gt'],
    aliases=[
        'Nom :', 'Nombre de candidats présents :', 'Taux de réussite :', 'Taux de mention',
        'Indice de position sociale :'
    ],
    localize=True
)
folium.GeoJson(
    lycees_75,
    tooltip=tooltip, 
    marker=folium.Marker(icon=folium.Icon(color="blue")),
).add_to(m)

m

## Modélisation

## Conclusion

### Perspectives